In [142]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import psycopg2 as psy
import numpy as np
import tqdm
from datetime import date


In [162]:
data = pd.read_csv("tblData.csv")#lee la base de datos
data = data.loc[data.IDDataTypeSub1 == "CALIBRATED"] #toma solo los valores calibrados

data = data.iloc[-100:] #toma las últimas 100 entradas (para disminuir el trabajo de datos)

In [163]:
data

,IDData,Version,IDDevice,IDDataType,IDDataTypeSub1,IDDataTypeSub2,IDDataTypeSub3,MethodName,IDMethodType,DateTime,...,Comment,CommentSub1,CommentSub2,CommentSub3,RecordType,Data,IDDataMaster,Attributes,Mission,MissionSub
22534,12FA_2019-09-27_10-00-00_000_228,1,SAM_8114,SPECTRUM,CALIBRATED,NaN,NaN,SAM_8114,SAM Control,09/27/19 10:00:00,...,NaN,NaN,NaN,NaN,0,0 5 0 0\r\n 310.036257132668 34.4396717405028...,NaN,CalFactor = 1\r\nIDBasisSpec = \r\nIDDataBack ...,No Mission,1
22536,12FA_2019-09-27_11-00-00_000_231,1,SAM_8114,SPECTRUM,CALIBRATED,NaN,NaN,SAM_8114,SAM Control,09/27/19 11:00:00,...,NaN,NaN,NaN,NaN,0,0 5 0 0\r\n 310.036257132668 31.1348507206212...,NaN,CalFactor = 1\r\nIDBasisSpec = \r\nIDDataBack ...,No Mission,1
22538,12FA_2019-09-27_12-00-00_000_234,1,SAM_8114,SPECTRUM,CALIBRATED,NaN,NaN,SAM_8114,SAM Control,09/27/19 12:00:00,...,NaN,NaN,NaN,NaN,0,0 5 0 0\r\n 310.036257132668 35.0935368377094...,NaN,CalFactor = 1\r\nIDBasisSpec = \r\nIDDataBack ...,No Mission,1
22540,12FA_2019-09-27_13-00-00_000_237,1,SAM_8114,SPECTRUM,CALIBRATED,NaN,NaN,SAM_8114,SAM Control,09/27/19 13:00:00,...,NaN,NaN,NaN,NaN,0,0 5 0 0\r\n 310.036257132668 40.1824754799691...,NaN,CalFactor = 1\r\nIDBasisSpec = \r\nIDDataBack ...,No Mission,1
22542,12FA_2019-09-27_14-00-00_000_240,1,SAM_8114,SPECTRUM,CALIBRATED,NaN,NaN,SAM_8114,SAM Control,09/27/19 14:00:00,...,NaN,NaN,NaN,NaN,0,0 6 0 0\r\n 310.036257132668 15.6673742527028...,NaN,CalFactor = 1\r\nIDBasisSpec = \r\nIDDataBack ...,No Mission,1
22544,12FA_2019-09-27_15-00-00_000_243,1,SAM_8114,SPECTRUM,CALIBRATED,NaN,NaN,SAM_8114,SAM Control,09/27/19 15:00:00,...,NaN,NaN,NaN,NaN,0,0 5 0 0\r\n 310.036257132668 26.4475720666456...,NaN,CalFactor = 1\r\nIDBasisSpec = \r\nIDDataBack ...,No Mission,1
22546,12FA_2019-09-27_16-00-00_000_246,1,SAM_8114,SPECTRUM,CALIBRATED,NaN,NaN,SAM_8114,SAM Control,09/27/19 16:00:00,...,NaN,NaN,NaN,NaN,0,0 6 0 0\r\n 310.036257132668 12.7632791281093...,NaN,CalFactor = 1\r\nIDBasisSpec = \r\nIDDataBack ...,No Mission,1
22548,12FA_2019-09-27_17-00-00_000_249,1,SAM_8114,SPECTRUM,CALIBRATED,NaN,NaN,SAM_8114,SAM Control,09/27/19 17:00:00,...,NaN,NaN,NaN,NaN,0,0 6 0 0\r\n 310.036257132668 10.8278384403777...,NaN,CalFactor = 1\r\nIDBasisSpec = \r\nIDDataBack ...,No Mission,1
22550,12FA_2019-09-27_18-00-00_000_252,1,SAM_8114,SPECTRUM,CALIBRATED,NaN,NaN,SAM_8114,SAM Control,09/27/19 18:00:00,...,NaN,NaN,NaN,NaN,0,0 7 0 0\r\n 310.036257132668 4.96882484940951...,NaN,CalFactor = 1\r\nIDBasisSpec = \r\nIDDataBack ...,No Mission,1
22552,12FA_2019-09-27_19-00-00_000_255,1,SAM_8114,SPECTRUM,CALIBRATED,NaN,NaN,SAM_8114,SAM Control,09/27/19 19:00:00,...,NaN,NaN,NaN,NaN,0,0 9 0 0\r\n 310.036257132668 1.83278792180816...,NaN,CalFactor = 1\r\nIDBasisSpec = \r\nIDDataBack ...,No Mission,1


In [105]:
def radiator():
    """selecciona los datos de la base de datos y retorna dos listas: máxima radiacion y radiación media"""
    listamax = []
    listamean = []
    
    for datos in data.Data:
   
        datos = re.split('0\r\n',datos)
        corr = [re.split("\s", value)[1:-2] for value in datos]
        rad = [float(value[0]) for value in corr]
        valor = [float(value[1]) for value in corr]
        
        maxrad = max(valor)
        listamax.append(maxrad)
        
        meanrad = np.mean(valor)
        listamean.append(meanrad)

    return listamax, listamean
 
radmax, radmean = radiator()

In [88]:
for fecha in data.IDData:
    data.replace(fecha,fecha[5:-11], inplace = True)


In [89]:
fecha = data["IDData"]


In [90]:
fecha = pd.to_datetime(fecha, format = '%Y-%m-%d_%H-%M')

In [147]:
today = pd.to_datetime(date.today())#dia actual
today

Timestamp('2019-10-07 00:00:00')

In [148]:
df = {'fecha':fecha, 'radmax':radmax, 'radmean':radmean}
df = pd.DataFrame(df)
df = df.loc[df.fecha > today]

In [ ]:
#POSTGRESSQL

In [138]:
def create_table():
    #connecting to the postgressql server (20 mb free)
    conn = None
    try:
        conn = psy.connect(host="salt.db.elephantsql.com", 
                           user="aeaudjwo", password="5uHoUq62gU5BoS2HvJQH0ogSw_94nxCE")

        c = conn.cursor()

        c.execute("""CREATE TABLE radiation(
                    fecha VARCHAR(255) UNIQUE,
                    maxrad REAL,
                    meanrad REAL)""")
        conn.commit()
        conn.close()
        
    except (Exception, psy.DatabaseError) as error:
        print(error)
        
    finally:
        if conn is not None:
            conn.close()
            
        


    

In [139]:
create_table()

In [140]:
def insert_data(fecha,maxrad,meanrad):
    conn = None
    try:
        conn = psy.connect(host="salt.db.elephantsql.com", 
                           user="aeaudjwo", password="5uHoUq62gU5BoS2HvJQH0ogSw_94nxCE")

        c = conn.cursor()
        
        c.execute("""INSERT INTO radiation (
                    fecha,
                    maxrad,
                    meanrad) VALUES (%s,%s,%s)""",(fecha,maxrad,meanrad))
        conn.commit()
        conn.close()
        
    except (Exception, psy.DatabaseError) as error:
        print (error)
    
    finally:
        if conn is not None:
            conn.close()


In [141]:
#inserta los valores nuevos
for f,mxr,mnr in zip(df.fecha, df.radmax, df.radmean):
    insert_data(f,mxr,mnr)

duplicate key value violates unique constraint "radiation_fecha_key"
DETAIL:  Key (fecha)=(2019-09-06 09:10:00) already exists.

duplicate key value violates unique constraint "radiation_fecha_key"
DETAIL:  Key (fecha)=(2019-09-06 09:10:00) already exists.

duplicate key value violates unique constraint "radiation_fecha_key"
DETAIL:  Key (fecha)=(2019-09-06 09:15:00) already exists.

duplicate key value violates unique constraint "radiation_fecha_key"
DETAIL:  Key (fecha)=(2019-09-06 09:16:00) already exists.

duplicate key value violates unique constraint "radiation_fecha_key"
DETAIL:  Key (fecha)=(2019-09-06 09:16:00) already exists.

duplicate key value violates unique constraint "radiation_fecha_key"
DETAIL:  Key (fecha)=(2019-09-06 09:16:00) already exists.

duplicate key value violates unique constraint "radiation_fecha_key"
DETAIL:  Key (fecha)=(2019-09-06 09:16:00) already exists.

